In [1]:
from neural_network import *
from activation_functions import *
from loss_functions import *
from layers import *
from model import *

from itertools import product as iter_product


In [2]:
def experiment(momentum_coefficient, learning_rate, learning_rate_half_life):
    num_input_neurons = 2
    num_output_neurons = 2

    def network_factory_one_neuron():
        return Neural_Network_Momentum(
            num_input_neurons = num_input_neurons,
            loss_function = MC_MSE(),
            batch_size = 50,
            learning_rate = learning_rate,
            layers = [
                Neural_Layer_Momentum(num_output_neurons, Identity_Function()),
            ],
            momentum_coefficient=momentum_coefficient
        )\
            .set_full_validation(False)\
            .compile()

    target_network_one_neuron = network_factory_one_neuron()
    training_network_one_neuron = network_factory_one_neuron()


    total_dataset_size_one_neuron = 10**5
    data_set_X_one_neuron = np.random.uniform(
        low = -1.0,
        high = 1.0,
        size = (num_input_neurons, total_dataset_size_one_neuron)
    )

    pre_train_loss = -1
    tries = 0
    target_starting_error = 0.1
    while pre_train_loss < target_starting_error and tries < 100:
        tries += 1
        target_network_one_neuron.reset_parameters()
        data_set_Y_one_neuron = target_network_one_neuron.make_predicitons(data_set_X_one_neuron)

        data_set_one_neuron = data_set_one_neuron = [
            data_set_X_one_neuron.reshape((total_dataset_size_one_neuron, num_input_neurons)), 
            data_set_Y_one_neuron.reshape((total_dataset_size_one_neuron, num_input_neurons)), 
        ]

        model_one_neuron = ML_Model(
            neural_network=training_network_one_neuron,
            whole_dataset_raw=data_set_one_neuron,
            learning_rate_half_life = learning_rate_half_life,            
            validation_batch_frequency=50,
        )

        pre_train_loss = model_one_neuron.test()



    epochs, _, learning_rates = model_one_neuron.train(epochs=25)

    post_train_loss = model_one_neuron.test()

    return epochs, learning_rates[-1], pre_train_loss, post_train_loss

In [3]:
experimental_momentum_coefficients = [
    0.0, 
    0.25,
    0.5,
    0.75,
    0.9,
]

experimental_learning_rates = [float(10**-i) for i in range(0, 5)]

experimental_learning_rate_half_lifes = [int(10**i) for i in range(0, 7)]

In [4]:
experimental_learning_rate_half_lifes

[1, 10, 100, 1000, 10000, 100000, 1000000]

Learning rate half life claculation is wrong, higher halflife for same epochs causes final learning rate to be smaller not higher

In [5]:
experiment_iterator = iter_product(
    experimental_momentum_coefficients, experimental_learning_rates, experimental_learning_rate_half_lifes
)

for experiment_i, (momentum, learning_rate, learning_rate_half_life) in enumerate(experiment_iterator):

    print(f"Experiment {experiment_i}:")

    experiment_parameters = dict(
        momentum_coefficient=momentum, 
        learning_rate = learning_rate, 
        learning_rate_half_life = learning_rate_half_life
    )

    print(f"experiment_parameters: {experiment_parameters}")

    try:
        epochs, final_learning_rate, pre_train_loss, post_train_loss = experiment(
            momentum_coefficient=momentum, 
            learning_rate = learning_rate, 
            learning_rate_half_life = learning_rate_half_life
        )
    except DiverganceError:
        print(f"Experiment failed divergance error")
    # except ParameterChangeZero:
    #     print(f"Experiment failed parameter change 0")
    else:
        print(f"Experiment success: in {epochs} epochs  with a final learning rate of {final_learning_rate} had a loss reducuction of {pre_train_loss} to {post_train_loss}")


Experiment 0:
experiment_parameters: {'momentum_coefficient': 0.0, 'learning_rate': 1.0, 'learning_rate_half_life': 1}
Experiment success: in 2 epochs  with a final learning rate of 0.9930924954370359 had a loss reducuction of 0.21188169793049347 to 0.22326421869898214
Experiment 1:
experiment_parameters: {'momentum_coefficient': 0.0, 'learning_rate': 1.0, 'learning_rate_half_life': 10}
Experiment success: in 2 epochs  with a final learning rate of 0.8705505632961241 had a loss reducuction of 0.659666862499755 to 0.3255977469543466
Experiment 2:
experiment_parameters: {'momentum_coefficient': 0.0, 'learning_rate': 1.0, 'learning_rate_half_life': 100}
Experiment success: in 2 epochs  with a final learning rate of 0.03125 had a loss reducuction of 0.3891425817873865 to 0.1835359581217943
Experiment 3:
experiment_parameters: {'momentum_coefficient': 0.0, 'learning_rate': 1.0, 'learning_rate_half_life': 1000}
Experiment success: in 2 epochs  with a final learning rate of 8.881784197001252e